In [4]:
# Load csv Dataset 
df = spark.read.csv('s3://ssotombigdata/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv', inferSchema=True, header=True)

print("Casos confirmados:", df.count())

Casos confirmados: 6211


In [5]:
#printSchema
df.printSchema()

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)



In [6]:
#drop some columns
df = df.drop('Codigo DIVIPOLA', 'FIS', 'fecha reporte web');
#printSchema
df.printSchema()

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)



In [7]:
#filter the multiple conditions
print("Recuperados en Medellín")
df.filter((df['Ciudad de ubicación']=='Medellín')&(df['atención']=='Recuperado')).select('ID de caso', 'Edad', 'Fecha recuperado', 'País de procedencia').show()

Recuperados en Medellín
+----------+----+--------------------+--------------------+
|ID de caso|Edad|    Fecha recuperado| País de procedencia|
+----------+----+--------------------+--------------------+
|         3|  50|2020-03-15T00:00:...|              España|
|         4|  55|2020-03-26T00:00:...|            Colombia|
|         5|  25|2020-03-23T00:00:...|            Colombia|
|        20|  26|2020-03-24T00:00:...|            Colombia|
|        21|  28|2020-04-10T00:00:...|            Colombia|
|        32|  55|2020-03-25T00:00:...|              España|
|       106|  44|2020-03-28T00:00:...|              España|
|       107|  56|2020-03-30T00:00:...|              España|
|       108|  57|2020-03-28T00:00:...|              España|
|       131|  22|2020-03-28T00:00:...|              España|
|       135|  44|2020-04-01T00:00:...|            Alemania|
|       136|  37|2020-04-13T00:00:...|            Colombia|
|       141|  62|2020-03-25T00:00:...|             Croacia|
|       142|  35

In [8]:
#group by
print("Número de casos por departamento")
df.groupBy('Departamento o Distrito ').count().orderBy('count', ascending=False).show(10, False)

Número de casos por departamento
+------------------------+-----+
|Departamento o Distrito |count|
+------------------------+-----+
|Bogotá D.C.             |2543 |
|Valle del Cauca         |877  |
|Antioquia               |468  |
|Meta                    |363  |
|Cartagena D.T. y C.     |246  |
|Cundinamarca            |219  |
|Risaralda               |193  |
|Barranquilla D.E.       |141  |
|Santa Marta D.T. y C.   |139  |
|Nariño                  |118  |
+------------------------+-----+
only showing top 10 rows



In [0]:
#target directory 
write_uri='s3://ssotombigdata/datasets/tmp/covid_out'

In [0]:
#save the dataframe as single csv 
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)